In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/'CodeTranslation '/'data (1)'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CodeTranslation /data (1)


Data Preprocessing - Code

In [3]:
def load_jsonl(input_path):
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

# use the function
data = load_jsonl('final.jsonl')

In [4]:
print(data[0]['java'][0])

class Solution { public TreeNode convertBST ( TreeNode root ) { int sum = 0 ; TreeNode node = root ; Stack < TreeNode > stack = new Stack < TreeNode > ( ) ; while ( ! stack . isEmpty ( ) || node != null ) { while ( node != null ) { stack . add ( node ) ; node = node . right ; } node = stack . pop ( ) ; sum += node . val ; node . val = sum ; node = node . left ; } return root ; } }


In [5]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


In [6]:
MAX_LENGTH = 200

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def readLangs(lang1, lang2, reverse =False):
    data = load_jsonl('train.jsonl')
    pairs = []
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)


    pairs = [(i['java'][0], i['python'][0]) for i in data]
    pairs = filterPairs(pairs)

    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])

    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)

    return input_lang, output_lang, pairs


def prepareData(lang1, lang2, reverse=False):
    return readLangs(lang1, lang2, reverse)



input_lang, output_lang, pairs = prepareData('java', 'python', True)


Counted words:
java 5586
python 4991


In [11]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [12]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [13]:

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [14]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    # print("indexes: " + str(indexes))
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [15]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [16]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [17]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [18]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [19]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [20]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [21]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 5000, print_every=50)

0m 16s (- 27m 25s) (50 1%) 43.9183
0m 29s (- 24m 0s) (100 2%) 54.6903
0m 41s (- 22m 11s) (150 3%) 56.4347
0m 53s (- 21m 35s) (200 4%) 52.2965
1m 5s (- 20m 52s) (250 5%) 47.6491
1m 19s (- 20m 45s) (300 6%) 61.1183
1m 32s (- 20m 23s) (350 7%) 55.4387
1m 45s (- 20m 8s) (400 8%) 63.3007
1m 58s (- 19m 54s) (450 9%) 59.9585
2m 9s (- 19m 26s) (500 10%) 48.5346
2m 20s (- 18m 54s) (550 11%) 55.7634
2m 34s (- 18m 53s) (600 12%) 60.0185
2m 46s (- 18m 32s) (650 13%) 50.6364
2m 59s (- 18m 22s) (700 14%) 66.4621
3m 12s (- 18m 12s) (750 15%) 64.9745
3m 24s (- 17m 51s) (800 16%) 52.5982
3m 36s (- 17m 36s) (850 17%) 64.0733
3m 48s (- 17m 21s) (900 18%) 50.8614
4m 0s (- 17m 3s) (950 19%) 48.1461
4m 12s (- 16m 50s) (1000 20%) 54.3326
4m 26s (- 16m 42s) (1050 21%) 56.1251
4m 39s (- 16m 29s) (1100 22%) 65.3391
4m 51s (- 16m 16s) (1150 23%) 53.5546
5m 5s (- 16m 5s) (1200 24%) 52.8114
5m 17s (- 15m 52s) (1250 25%) 52.0941
5m 30s (- 15m 40s) (1300 26%) 54.8820
5m 42s (- 15m 26s) (1350 27%) 61.4648
5m 54s (- 1

In [22]:
evaluateRandomly(encoder1, attn_decoder1)

> import java . util . * ; public class Main { public static void main ( String [ ] args ) { Scanner sc = new Scanner ( System . in ) ; int x = sc . nextInt ( ) ; long count = 1 ; while ( true ) { long d = count * ( count + 1 ) / 2 ; if ( d >= x ) { System . out . println ( count ) ; return ; } count ++ ; } } }
= x = int ( input ( ) ) NEW_LINE for i in range ( x + 1 ) : NEW_LINE INDENT if ( i ** 2 + i ) // 2 >= x : NEW_LINE INDENT print ( i ) NEW_LINE break NEW_LINE DEDENT DEDENT
< class NEW_LINE % NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE N

In [23]:
def load_jsonl(input_path):
    data1 = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data1.append(json.loads(line.strip()))
    return data1

# use the function
data1 = load_jsonl('test.jsonl')

In [24]:
print(data1[129]['java'])
print(data1[129]['python'])

['public class Test { public static int findMajority ( int arr [ ] , int n ) { return arr [ n / 2 ] ; } public static void main ( String args [ ] ) { int arr [ ] = { 1 , 2 , 2 , 3 } ; int n = arr . length ; System . out . println ( findMajority ( arr , n ) ) ; } }']
['def findMajority ( arr , n ) : NEW_LINE INDENT return arr [ int ( n / 2 ) ] NEW_LINE DEDENT arr = [ 1 , 2 , 2 , 3 ] NEW_LINE n = len ( arr ) NEW_LINE print ( findMajority ( arr , n ) ) NEW_LINE']


In [25]:
for i in range(len(data1)):
  try:
    output_words, attentions = evaluate(encoder1, attn_decoder1, data1[i]['java'][0])
    output_sentence = ' '.join(output_words)
    print(i, output_sentence)
  except:
    continue

4 def ) NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NE

In [26]:
print(data1[172]['java'][0])
output_words, attentions = evaluate(encoder1, attn_decoder1, data1[172]['java'][0])
output_sentence = ' '.join(output_words)
print(output_sentence)

class Test { static int factorial ( int n ) { int i = n , fact = 1 ; while ( n / i != n ) { fact = fact * i ; i -- ; } return fact ; } public static void main ( String [ ] args ) { int num = 5 ; System . out . println ( " Factorial ▁ of ▁ " + num + " ▁ is ▁ " + factorial ( 5 ) ) ; } }
def ) NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LINE NEW_LI